In [34]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from tensorflow.keras.applications import ResNet50V2
tf.__version__

'2.6.0-dev20210402'

In [5]:
PATH = 'data'

os.listdir(PATH)

['image',
 'sample_submission.csv',
 'subset.csv',
 'test.csv',
 'train.csv',
 'train_split.csv',
 'val_split.csv']

In [10]:
train = pd.read_csv(f'{PATH}/subset.csv')
val = pd.read_csv(f'{PATH}/val_split.csv')
train.shape, val.shape

((2220, 8), (10932, 8))

In [29]:
BATCH_SIZE = 64

def decode(name, label):
    img = tf.io.read_file(name)
    img = tf.image.decode_jpeg(img,channels=3)
    img = tf.cast(img, tf.float32)
    return img,label


def load_ds(df):
    imgs , labels = df["image_name"].values, df["target"].values
    imgs = [f'{PATH}/image/train/{name}.jpg' for name in imgs]
    ds = tf.data.Dataset.from_tensor_slices((imgs,labels))
    ds = ds.map(decode)
    ds = ds.shuffle(2048)
    ds = ds.batch(BATCH_SIZE)
    return ds

In [31]:
train_ds = load_ds(train)
val_ds = load_ds(train)

In [36]:
IMAGE_SIZE = (224,224,3)

encoder = ResNet50V2(
    include_top=False,
    input_shape=IMAGE_SIZE,
    weights='imagenet'
)
encoder.trainable=False

inputs = keras.Input(shape=IMAGE_SIZE)
x = keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = encoder(x, training = False)
x = keras.layers.GlobalAveragePooling2D()(x)
ouputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, ouputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
rescaling_1 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 7, 7, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_________________________________________________________________


In [37]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.AUC(name="auc")]
)

In [ ]:
model.fit(train_ds, epochs=10, validation_data=val_ds, validation_steps=10)